# Camada trusted
## Criação de características relevantes

Importando biblotecas necessárias

In [1]:
import pandas as pd

lendo base de dados original com reclamações do consumidor.gov em formato csv.

In [2]:
claims = pd.read_csv(
    filepath_or_buffer='../dataset/raw/consumidor_gov.csv',
    delimiter=',',
    quotechar='"',
    header=0,
    names=[
        'company_name',
        'date',
        'uf',
        'city',
        'customer_claim',
        'company_response'
    ],
)

Convertendo coluna `date` do tipo _string_ para o tipo _date_.

In [3]:
claims['date'] = claims['date'].astype('datetime64[ns]')

Removendo caracteres especias (`\n`, `\r`, `\t`) de início e fim de texto para as colunas `customer_claim` e `company_response`.

In [4]:
claims['customer_claim'] = claims['customer_claim'].apply(lambda text: text.strip())
claims['company_response'] = claims['company_response'].apply(lambda text: text.strip())

Substituindo caracteres especiais (`\n`, `\r`, `\t`) no meio do texto por espaço vazio para as colunas `customer_claim` e `company_response`.

In [5]:
claims['customer_claim'] = claims['customer_claim'].str.replace(
    pat='[\n\r\t]+',
    repl=' ',
    regex=True
)

In [6]:
claims['company_response'] = claims['company_response'].str.replace(
    pat='[\n\r\t]+',
    repl=' ',
    regex=True
)

Explicitando reclamações sem resposta com valor `None`.

In [7]:
claims['company_response'] =  claims['company_response'].replace({'<não respondido pela empresa>': None})

Removendo índice original do arquivo csv.

In [8]:
claims.reset_index(drop=True, inplace=True)

Visualizando $5$ primeiros registros.

In [10]:
claims.head()

,company_name,date,uf,city,customer_claim,company_response
0,Correios,2021-04-13,MG,São João Del Rei,"Boa tarde ! No dia 08/04/2021, por volta das 1...","Prezado Cliente, para que possamos verificar o..."
1,Samsung,2021-04-25,MG,Belo Horizonte,Fui realizar essa ultima atualização do androi...,"Caro cliente, Conforme contato realizado atr..."
2,Vivo - Telefônica,2021-04-14,MG,Caratinga,"Ao contatar a empresa, solicitei a 2ª via de c...",Prezada Sra. Vitoria Realizamos tentativa de c...
3,Vivo - Telefônica,2021-04-13,MG,Carmópolis de Minas,Fiz uma reclamação anteriormente a respeito de...,Prezada Sra. Juliana Realizamos tentativa de c...
4,Vivo - Telefônica,2021-04-14,MG,Santa Bárbara do Leste,Solicitei a Vivo plano para o número (33) XXXX...,Contatamos o responsável pela manifestação no ...


Analisando tipos de dados das colunas.

In [11]:
claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378574 entries, 0 to 378573
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   company_name      378574 non-null  object        
 1   date              378574 non-null  datetime64[ns]
 2   uf                378574 non-null  object        
 3   city              378574 non-null  object        
 4   customer_claim    378574 non-null  object        
 5   company_response  375107 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 17.3+ MB


Filtrando por reclamações do segmento de telofonia fixa e móvel

In [52]:
companies = [
    'Claro Fixo - Embratel',
    'Claro Celular',
    'Oi Fixo',
    'Oi Celular',
    'Vivo - Telefônica (GVT)',
    'Vivo - Telefônica',
    'Tim',
]

companies_mask = claims['company_name'].isin(companies)

In [53]:
telecom_seg_claims =  claims.loc[companies_mask]

Filtrando por período de tempo específico

In [54]:
start_date = '2020-01-01'
end_date = '2020-12-31'

date_mask = (claims['date'] >= start_date) & (claims['date'] <= end_date)

In [55]:
telecom_pandemic_claims =  telecom_seg_claims.loc[date_mask]

Visualizando dados

In [56]:
telecom_pandemic_claims.head()

,company_name,date,uf,city,customer_claim,company_response
42907,Vivo - Telefônica,2020-12-23,MG,Belo Horizonte,O TELEFONE ACIMA CITADO ESTAVA LIGADO À TIM. F...,"Curitiba, 13 de Dezembro de 2020. Protoc..."
43118,Tim,2020-12-30,MG,Belo Horizonte,"Bom dia, Recebi ontem, 29/12/2020, ligação do ...","Prezado Guilherme, boa tarde! Em atenção à su..."
43122,Tim,2020-12-31,MG,Governador Valadares,Usei os canais digitais para migrar meu plano ...,Prezado cliente Sua demanda foi concluída no ...
43123,Tim,2020-12-30,MG,Juiz de Fora,O plano foi cancelado no dia 26/11/2020 e poss...,"""Prezado Jeferson, Em atenção à sua demanda pe..."
43124,Tim,2020-12-30,MG,Lagoa Santa,Trabalho em uma região de Lagoa Santa/MG que o...,Prezado(a) Vinicius ! Em virtude do prazo esti...


Salvando todo o conjunto de dados tratado e transformado em parquet.

In [65]:
telecom_pandemic_claims.to_parquet(
    path='../dataset/trusted/trzd_telecom_pandemic_claims.parquet.snappy',
    compression='snappy',
    index=True,
)